In [1]:
from xgboost import plot_importance
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.api import Logit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score, roc_curve, auc
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_curve, auc,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

import scipy.stats
from scipy import stats
%matplotlib inline
matplotlib.rc('font',family = 'Malgun Gothic')

In [2]:
# 전처리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

# 머신러닝 알고리즘
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
import statsmodels.api as sm


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [6]:
df_raw = pd.read_csv("Dataset_all.csv")
df_raw.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
# 더미 변수화
df_raw[['Ox_Chamber']]=df_raw[['Ox_Chamber']].astype(str)
df_raw[['photo_soft_Chamber']]=df_raw[['photo_soft_Chamber']].astype(str)
df_raw[['lithography_Chamber']]=df_raw[['lithography_Chamber']].astype(str)
df_raw[['Etching_Chamber']]=df_raw[['Etching_Chamber']].astype(str)
df_raw[['Chamber_Num']]=df_raw[['Chamber_Num']].astype(str)
df_raw[['Wavelength']]=df_raw[['Wavelength']].astype(str)

# wet, dry 구분
# df_raw.loc[df_raw['type']=='dry','type']=0
# df_raw.loc[df_raw['type']=='wet','type']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Wavelength 365, 405, 436
df_raw.loc[df_raw['Wavelength']=='365','Wavelength']=0
df_raw.loc[df_raw['Wavelength']=='405','Wavelength']=1
df_raw.loc[df_raw['Wavelength']=='436','Wavelength']=2


# 필요 없는 변수 material, current, lamp, chamber, type제거
# df_list_before=['Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time', 'thickness',
#         'resist_target', 'N2_HMDS', 'pressure_HMDS', 'temp_HMDS','temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake', 'temp_softbake', 'time_softbake',
#         'Line_CD','Wavelength', 'Resolution', 'Energy_Exposure',
#         'Thin F4','Thin F2','Thin F3','Thin F1', 'Temp_Etching','Source_Power', 'Selectivity',
#         'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s','Flux840s', 'input_Energy', 'Temp_implantation','Furance_Temp', 'RTA_Temp']

# df_dummy=pd.get_dummies(df_raw[df_list_before])

# # 불량판단 기준 달라질 것. 이 파라미터에 따른 성능 생각할 필요
# Y_wet.loc[Y_wet.Target<183]=0
# Y_wet.loc[Y_wet.Target>=183]=1
# Y_dry.loc[Y_dry.Target<183]=0
# Y_dry.loc[Y_dry.Target>=183]=1
# len(X),len(Y)
# df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(X,Y,test_size = 0.3,random_state = 1234)

In [8]:
df_raw.loc[df_raw.Target>=183,'Quality']=1
df_raw.loc[df_raw.Target<183,'Quality']=0

In [9]:
# 제일 좋은 것 같은데
imp_features=['Temp_OXid', 'Thin F4', 'Thin F2', 'Flux480s', 'Source_Power',
        'thickness', 'Pressure', 'Energy_Exposure',
       'photoresist_bake', 'spin2', 'temp_HMDS_bake','Temp_implantation']
df_dummy=pd.get_dummies(df_raw[imp_features])
# ppm 제거

In [10]:
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_dummy,df_raw['Quality'],test_size = 0.3,random_state = 1234)

In [11]:
df_test_x

,Temp_OXid,Thin F4,Thin F2,Flux480s,Source_Power,thickness,Pressure,Energy_Exposure,photoresist_bake,spin2,temp_HMDS_bake,Temp_implantation
326,1267.846723,224.0,3645.0,2.982513e+17,50.431,703.950,0.217,106.328,5.063,4049.123,202.456,101.078
771,871.477974,174.0,3635.0,2.993495e+17,51.358,709.835,0.213,108.407,5.065,4074.581,203.729,103.396
288,1159.860168,387.0,3658.0,2.982403e+17,50.787,706.277,0.202,111.001,5.004,4036.214,205.521,100.362
230,1282.303070,175.0,3646.0,2.993413e+17,50.185,716.564,0.215,105.132,5.021,4017.361,200.868,100.463
362,1280.077304,177.0,3634.0,3.014882e+17,50.792,709.301,0.216,106.102,5.103,4103.032,205.152,101.980
...,...,...,...,...,...,...,...,...,...,...,...,...
137,1270.342348,361.0,3661.0,3.006096e+17,51.138,724.096,0.223,107.722,4.934,3944.064,197.203,102.845
9,1269.092800,178.0,3648.0,3.002527e+17,50.839,721.528,0.217,107.212,5.016,3998.147,199.907,102.099
77,871.800986,326.0,3653.0,3.006004e+17,51.904,715.604,0.219,108.335,4.986,3987.046,199.352,104.759
475,870.794989,183.0,3615.0,2.987389e+17,50.559,690.969,0.192,107.851,5.110,4100.279,205.014,101.397


In [12]:
df_test_x_1 = pd.DataFrame()

In [13]:
df_test_x_1 = df_test_x.copy()

In [14]:
range_1 = [] #Temp_Oxid
range_2 = [] #Pressure
range_3 = [] #temp_HMDS_bake
range_4 = [] #photoresist_bake
range_5 = [] #spin2
range_6 = [] #Temp_implantation

In [15]:
for i in range(1, 250):
    range_1.append(random.uniform(1282.139,1310.709))
    range_2.append(random.uniform(0.180,0.193))
    range_3.append(random.uniform(207.765,209.521))
    range_4.append(random.uniform(1282.139,1310.709))
    range_5.append(random.uniform(4105.862,4208.676))
    range_6.append(random.uniform(97.744,100.22))

In [16]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [17]:
df_test_x_1[['Temp_OXid','Pressure','temp_HMDS_bake','photoresist_bake','spin2','Temp_implantation']]

,Temp_OXid,Pressure,temp_HMDS_bake,photoresist_bake,spin2,Temp_implantation
326,1267.846723,0.217,202.456,5.063,4049.123,101.078
771,871.477974,0.213,203.729,5.065,4074.581,103.396
288,1159.860168,0.202,205.521,5.004,4036.214,100.362
230,1282.303070,0.215,200.868,5.021,4017.361,100.463
362,1280.077304,0.216,205.152,5.103,4103.032,101.980
72,1103.547945,0.210,202.072,5.107,3999.030,102.933
415,1287.435437,0.203,197.046,5.067,4010.001,103.891
301,1130.769716,0.196,204.080,4.964,4071.649,103.749
746,1274.667489,0.224,196.262,4.904,3925.230,103.090
406,1054.787096,0.210,200.051,5.076,4052.404,105.080


In [18]:
range1 = pd.DataFrame()
range2 = pd.DataFrame()
range3 = pd.DataFrame()
range4 = pd.DataFrame()
range5 = pd.DataFrame()
range6 = pd.DataFrame()

In [19]:
range1 = range_1
range2 = range_2
range3 = range_3
range4 = range_4
range5 = range_5
range6 = range_6

In [20]:
df_test_x_1.columns

Index(['Temp_OXid', 'Thin F4', 'Thin F2', 'Flux480s', 'Source_Power',
       'thickness', 'Pressure', 'Energy_Exposure', 'photoresist_bake', 'spin2',
       'temp_HMDS_bake', 'Temp_implantation'],
      dtype='object')

In [21]:
df_test_x_1['Temp_OXid'] = range1
df_test_x_1['Pressure'] = range2
df_test_x_1['temp_HMDS_bake'] = range3
df_test_x_1['photoresist_bake'] = range4
df_test_x_1['spin2'] = range5
df_test_x_1['Temp_implantation'] = range5

In [22]:
df_test_x_1[['Temp_OXid','Pressure','temp_HMDS_bake','photoresist_bake','spin2','Temp_implantation']]

,Temp_OXid,Pressure,temp_HMDS_bake,photoresist_bake,spin2,Temp_implantation
326,1298.673995,0.183527,208.762489,1293.306623,4187.906411,4187.906411
771,1289.691013,0.181625,209.034325,1301.510718,4137.469703,4137.469703
288,1288.544431,0.182742,209.034238,1294.471336,4149.686418,4149.686418
230,1304.719067,0.188451,208.802718,1284.053884,4188.528656,4188.528656
362,1303.477061,0.189863,208.634411,1286.794699,4127.819516,4127.819516
72,1309.905931,0.188617,208.053640,1309.740764,4147.738602,4147.738602
415,1284.436680,0.187513,208.442452,1290.377984,4201.395069,4201.395069
301,1308.550589,0.188481,207.952944,1289.401714,4160.353739,4160.353739
746,1293.005284,0.187123,209.439051,1299.991940,4107.502811,4107.502811
406,1284.214040,0.185487,209.441704,1299.151188,4202.689269,4202.689269


In [23]:
df_test_x_1

,Temp_OXid,Thin F4,Thin F2,Flux480s,Source_Power,thickness,Pressure,Energy_Exposure,photoresist_bake,spin2,temp_HMDS_bake,Temp_implantation
326,1298.673995,224.0,3645.0,2.982513e+17,50.431,703.950,0.183527,106.328,1293.306623,4187.906411,208.762489,4187.906411
771,1289.691013,174.0,3635.0,2.993495e+17,51.358,709.835,0.181625,108.407,1301.510718,4137.469703,209.034325,4137.469703
288,1288.544431,387.0,3658.0,2.982403e+17,50.787,706.277,0.182742,111.001,1294.471336,4149.686418,209.034238,4149.686418
230,1304.719067,175.0,3646.0,2.993413e+17,50.185,716.564,0.188451,105.132,1284.053884,4188.528656,208.802718,4188.528656
362,1303.477061,177.0,3634.0,3.014882e+17,50.792,709.301,0.189863,106.102,1286.794699,4127.819516,208.634411,4127.819516
72,1309.905931,456.0,3638.0,2.981188e+17,51.156,716.988,0.188617,107.687,1309.740764,4147.738602,208.053640,4147.738602
415,1284.436680,304.0,3625.0,2.971368e+17,51.062,714.203,0.187513,106.617,1290.377984,4201.395069,208.442452,4201.395069
301,1308.550589,448.0,3654.0,2.985401e+17,51.969,726.396,0.188481,109.447,1289.401714,4160.353739,207.952944,4160.353739
746,1293.005284,169.0,3668.0,2.999457e+17,51.236,726.633,0.187123,107.961,1299.991940,4107.502811,209.439051,4107.502811
406,1284.214040,366.0,3654.0,3.013019e+17,50.583,697.237,0.185487,107.421,1299.151188,4202.689269,209.441704,4202.689269


In [24]:
df_test_x

,Temp_OXid,Thin F4,Thin F2,Flux480s,Source_Power,thickness,Pressure,Energy_Exposure,photoresist_bake,spin2,temp_HMDS_bake,Temp_implantation
326,1267.846723,224.0,3645.0,2.982513e+17,50.431,703.950,0.217,106.328,5.063,4049.123,202.456,101.078
771,871.477974,174.0,3635.0,2.993495e+17,51.358,709.835,0.213,108.407,5.065,4074.581,203.729,103.396
288,1159.860168,387.0,3658.0,2.982403e+17,50.787,706.277,0.202,111.001,5.004,4036.214,205.521,100.362
230,1282.303070,175.0,3646.0,2.993413e+17,50.185,716.564,0.215,105.132,5.021,4017.361,200.868,100.463
362,1280.077304,177.0,3634.0,3.014882e+17,50.792,709.301,0.216,106.102,5.103,4103.032,205.152,101.980
72,1103.547945,456.0,3638.0,2.981188e+17,51.156,716.988,0.210,107.687,5.107,3999.030,202.072,102.933
415,1287.435437,304.0,3625.0,2.971368e+17,51.062,714.203,0.203,106.617,5.067,4010.001,197.046,103.891
301,1130.769716,448.0,3654.0,2.985401e+17,51.969,726.396,0.196,109.447,4.964,4071.649,204.080,103.749
746,1274.667489,169.0,3668.0,2.999457e+17,51.236,726.633,0.224,107.961,4.904,3925.230,196.262,103.090
406,1054.787096,366.0,3654.0,3.013019e+17,50.583,697.237,0.210,107.421,5.076,4052.404,200.051,105.080


In [25]:
xgb_model = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=5, max_leaves=1, min_child_weight=1,
                               monotone_constraints='()',
                               n_estimators=100, n_jobs=4, num_parallel_tree=1,
                               process_type='default', random_state=1234,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=0.8, tree_method='exact',
                               validate_parameters=1, verbosity=None)

In [26]:
accuracy_test=[]
precision=[]
recall=[]
F1=[]

## XGB

In [27]:
xgb_model.fit(df_train_x,df_train_y)

[16:22:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              max_leaves=1, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, process_type='default', random_state=1234,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
y_pred = xgb_model.predict_proba(df_test_x)
threshold=0.2
y_pred[:,1][y_pred[:,1]>=threshold]=1
y_pred[:,1][y_pred[:,1]<threshold]=0
# print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[217   7]
 [ 10  15]]
0.9317269076305221
0.6818181818181818
0.6
0.6382978723404256


In [29]:
y_pred

array([[9.9984688e-01, 0.0000000e+00],
       [9.9976099e-01, 0.0000000e+00],
       [9.9754041e-01, 0.0000000e+00],
       [9.9954146e-01, 0.0000000e+00],
       [9.9859226e-01, 0.0000000e+00],
       [9.9907291e-01, 0.0000000e+00],
       [9.9987739e-01, 0.0000000e+00],
       [9.9361527e-01, 0.0000000e+00],
       [9.9804282e-01, 0.0000000e+00],
       [9.9754858e-01, 0.0000000e+00],
       [9.3854022e-01, 0.0000000e+00],
       [9.9800569e-01, 0.0000000e+00],
       [9.9920934e-01, 0.0000000e+00],
       [6.8588245e-01, 1.0000000e+00],
       [2.7404344e-01, 1.0000000e+00],
       [9.9979955e-01, 0.0000000e+00],
       [9.9729830e-01, 0.0000000e+00],
       [9.9935216e-01, 0.0000000e+00],
       [9.9931329e-01, 0.0000000e+00],
       [9.9865288e-01, 0.0000000e+00],
       [9.9970138e-01, 0.0000000e+00],
       [9.9206686e-01, 0.0000000e+00],
       [9.9930173e-01, 0.0000000e+00],
       [9.9780869e-01, 0.0000000e+00],
       [9.9975544e-01, 0.0000000e+00],
       [9.9899906e-01, 0.

In [30]:
y_pred_f = xgb_model.predict_proba(df_test_x_1)
threshold=0.2
y_pred_f[:,1][y_pred_f[:,1]>=threshold]=1
y_pred_f[:,1][y_pred_f[:,1]<threshold]=0
# print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred_f[:,1])))
print(accuracy_score(df_test_y, y_pred_f[:,1]))
print(precision_score(df_test_y, y_pred_f[:,1]))
print(recall_score(df_test_y, y_pred_f[:,1]))
print(f1_score(df_test_y, y_pred_f[:,1]))

Confustion matrix : 
[[213  11]
 [ 14  11]]
0.8995983935742972
0.5
0.44
0.46808510638297873


In [31]:
y_pred_f

array([[0.9988066 , 0.        ],
       [0.9994994 , 0.        ],
       [0.8770466 , 0.        ],
       [0.9990762 , 0.        ],
       [0.99749786, 0.        ],
       [0.9876003 , 0.        ],
       [0.9994457 , 0.        ],
       [0.8297751 , 0.        ],
       [0.99949896, 0.        ],
       [0.9684064 , 0.        ],
       [0.9713779 , 0.        ],
       [0.9391872 , 0.        ],
       [0.9975712 , 0.        ],
       [0.3853221 , 1.        ],
       [0.32372713, 1.        ],
       [0.99793816, 0.        ],
       [0.9732553 , 0.        ],
       [0.99510133, 0.        ],
       [0.9966109 , 0.        ],
       [0.99586   , 0.        ],
       [0.99905175, 0.        ],
       [0.9230231 , 0.        ],
       [0.99775535, 0.        ],
       [0.9456934 , 0.        ],
       [0.9939633 , 0.        ],
       [0.9987448 , 0.        ],
       [0.99758977, 0.        ],
       [0.99949074, 0.        ],
       [0.9898816 , 0.        ],
       [0.9981713 , 0.        ],
       [0.

In [32]:
y = y_pred[:,1].astype(int)

In [33]:
y = list(y)
y.count(1)

22

In [34]:
y_ = y_pred_f[:,1].astype(int)

In [35]:

y_ = list(y_)
y_.count(1)

22

In [172]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## GB

In [173]:
gb_fianl=GradientBoostingClassifier(ccp_alpha=0.0,
                            criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance',
                            max_depth=3, max_features=None,
                            max_leaf_nodes=None,
                            min_impurity_decrease=0.0,
                            min_impurity_split=None,
                            min_samples_leaf=1,
                            min_samples_split=2,
                            min_weight_fraction_leaf=0.0,
                            n_estimators=100,
                            n_iter_no_change=None,
                            presort='deprecated',
                            random_state=None, subsample=1.0,
                            tol=0.0001, validation_fraction=0.1,
                            verbose=0, warm_start=False)
gb_fianl.fit(df_train_x,df_train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [174]:
y_pred = gb_fianl.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[212  12]
 [  8  17]]
0.9196787148594378
0.5862068965517241
0.68
0.6296296296296295


In [175]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## RF

In [176]:
rf_final=RandomForestClassifier(bootstrap=True,
                       ccp_alpha=0.0,
                       class_weight=None,
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       max_samples=None,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=100,
                       n_jobs=None,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False)
rf_final.fit(df_train_x,df_train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [177]:
y_pred = rf_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[202  22]
 [  3  22]]
0.8995983935742972
0.5
0.88
0.6376811594202899


In [178]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## DT

In [179]:
dt_final=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=4,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=20,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best')
dt_final.fit(df_train_x,df_train_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [180]:
y_pred = dt_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[204  20]
 [  2  23]]
0.9116465863453815
0.5348837209302325
0.92
0.6764705882352942


In [181]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## SVM

In [182]:
svm_final=SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=True, random_state=None, shrinking=True,
                     tol=0.001, verbose=False)
svm_final.fit(df_train_x,df_train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [183]:
y_pred = svm_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[186  38]
 [ 24   1]]
0.751004016064257
0.02564102564102564
0.04
0.03125


In [184]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## 로지스틱 회귀

In [185]:
df_raw_logit=df_raw
df_raw_logit.rename(columns = {'Thin F1' : 'Thin_F1','Thin F2' : 'Thin_F2','Thin F3' : 'Thin_F3','Thin F4' : 'Thin_F4'}, inplace = True)
df_train_logit, df_test_logit = train_test_split(df_raw_logit, test_size = 0.3, random_state = 1234)

In [186]:
# from_formula 함수를 이용하여 변수 역할 지정
# 분석 대상 이벤트 = “1”
log_model = Logit.from_formula("""Quality ~ Thin_F4 + Thin_F2 + Thin_F3 + Pressure + Flux90s +
 ppm""", df_train_logit)
# 적합
log_result = log_model.fit()
# 결과 출력
print(log_result.summary())

Optimization terminated successfully.
         Current function value: 0.151028
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                Quality   No. Observations:                  581
Model:                          Logit   Df Residuals:                      580
Method:                           MLE   Df Model:                            0
Date:                Thu, 18 Mar 2021   Pseudo R-squ.:                  0.5117
Time:                        11:59:10   Log-Likelihood:                -87.747
converged:                       True   LL-Null:                       -179.70
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -265.0517     47.530     -5.577      0.000    -358.209    -171.894
Thin_F4        0.0172      0.

In [187]:
y_pred = log_result.predict(df_test_logit)
th = 0.2
# 0과 1의 값을 가진 class로 변환
y_pred_class = (y_pred > th).astype(int)
# 실제 test 데이터의 목표변수와 test 데이터의 예측값 비교
print("Accuracy: {0:.3f}".format(accuracy_score(df_test_logit["Quality"], y_pred_class)))
print("f1: {0:.3f}".format(f1_score(df_test_logit["Quality"], y_pred_class)))
print("recall: {0:.3f}\n".format(recall_score(df_test_logit["Quality"], y_pred_class)))

print("Confusion Matrix: \n{}".format(confusion_matrix(df_test_logit["Quality"], y_pred_class)))

Accuracy: 0.859
f1: 0.507
recall: 0.720

Confusion Matrix: 
[[196  28]
 [  7  18]]


In [188]:
accuracy_test.append(accuracy_score(df_test_logit["Quality"], y_pred_class))
precision.append(precision_score(df_test_logit["Quality"], y_pred_class))
recall.append(recall_score(df_test_logit["Quality"], y_pred_class))
F1.append(f1_score(df_test_logit["Quality"], y_pred_class))

In [189]:
accuracy_test

[0.9317269076305221,
 0.9196787148594378,
 0.8995983935742972,
 0.9116465863453815,
 0.751004016064257,
 0.8594377510040161]

In [190]:
precision

[0.6818181818181818,
 0.5862068965517241,
 0.5,
 0.5348837209302325,
 0.02564102564102564,
 0.391304347826087]

In [191]:
recall

[0.6, 0.68, 0.88, 0.92, 0.04, 0.72]

In [192]:
F1

[0.6382978723404256,
 0.6296296296296295,
 0.6376811594202899,
 0.6764705882352942,
 0.03125,
 0.5070422535211268]

In [193]:
final_model=pd.DataFrame({'accuracy':accuracy_test,'precision':precision,'recall':recall,'F1':F1},index=['XGB','GB','RF','DT','SVM','Logistic'])

In [194]:
final_model.T


,XGB,GB,RF,DT,SVM,Logistic
accuracy,0.931727,0.919679,0.899598,0.911647,0.751004,0.859438
precision,0.681818,0.586207,0.500000,0.534884,0.025641,0.391304
recall,0.600000,0.680000,0.880000,0.920000,0.040000,0.720000
F1,0.638298,0.629630,0.637681,0.676471,0.031250,0.507042
